# 0. Directory Setting & Variables

In [1]:
import os
from xml.etree.ElementTree import parse
import numpy as np
import cv2
from tqdm import tqdm

In [5]:
original_dir = 'E:/data/original/' # 비디오/xml 파일을 불러올 경로
# frame_dir = 'E:/data/frame/' # frame을 저장할 경로
target_dir = 'E:/data/target/' # crop한 비디오를 저장할 경로

# 비디오 카테고리 지정 ('wander', 'fight', 'datefight')
cat_name = 'dump'
# 추출할 action 카테고리 지정 ('stop and go', 'punching')
action_name = 'drop'
# 추출할 frame 개수 지정 (위 MEMO 보고 action 종류에 따라 다르게 설정하기)
how_many_frame = 100

In [6]:
vid_lst = [] # video 리스트
xml_lst = [] # xml 리스트

for file in os.listdir(original_dir + 'original_' + cat_name):
    if 'mp4' in file:
        vid_lst.append(file)
    else:
        xml_lst.append(file)
        
vid_lst, xml_lst

(['162-3_cam01_dump02_place04_night_spring.mp4',
  '162-3_cam01_dump02_place04_night_summer.mp4',
  '162-3_cam02_dump02_place04_night_spring.mp4',
  '162-3_cam02_dump02_place04_night_summer.mp4',
  '162-4_cam01_dump02_place04_night_spring.mp4',
  '162-4_cam01_dump02_place04_night_summer.mp4',
  '162-4_cam02_dump02_place04_night_spring.mp4',
  '162-4_cam02_dump02_place04_night_summer.mp4',
  '162-5_cam01_dump02_place04_night_spring.mp4',
  '162-5_cam01_dump02_place04_night_summer.mp4',
  '162-5_cam02_dump02_place04_night_spring.mp4',
  '162-5_cam02_dump02_place04_night_summer.mp4',
  '162-6_cam01_dump02_place04_night_spring.mp4',
  '162-6_cam01_dump02_place04_night_summer.mp4',
  '162-6_cam02_dump02_place04_night_spring.mp4',
  '162-6_cam02_dump02_place04_night_summer.mp4',
  '163-1_cam01_dump02_place09_night_spring.mp4',
  '163-1_cam01_dump02_place09_night_summer.mp4',
  '163-1_cam02_dump02_place09_night_spring.mp4',
  '163-1_cam02_dump02_place09_night_summer.mp4',
  '163-2_cam01_dump0

# 1. Extract Frames and Save as a Video

In [7]:
# 1. EXTRACT VIDEO INFO

for video_name in tqdm(vid_lst):
    
    begin_frame = -1
    xml_name = original_dir + 'original_' + cat_name + '/' + video_name.replace(".mp4", ".xml")
#     print(video_name, xml_name)
    
    tree = parse(xml_name)
    root = tree.getroot() # root tag: annotation
    
    # video_len(초) 계산
    header = root.find("header")
    duration = header.find("duration").text.split(":")
    for item in duration:
        if duration[0] != '00': # duration이 1시간 이상일 경우
            video_len = float(duration[0])*3600 + float(duration[1])*60 + float(duration[-1])
        else: # duration이 1시간 미만일 경우
            video_len = float(duration[1])*60 + float(duration[-1])
    fps = int(header.find("fps").text)

    # action tag
    object_ = root.find("object")
    action = object_.findall("action")
    
    # 가장 첫 번째로 등장하는 action에 대해 frame index 추출
    for act in action:
        if act.find("actionname").text == action_name:
            frame_lst = act.findall("frame")
            for frame in frame_lst:
                begin_frame, end_frame = int(frame.find("start").text), int(frame.find("end").text)
                if end_frame - begin_frame >= how_many_frame: # 추출할 수 있는 frame 수가 100 이상인 경우에만 비디오에서 frame 추출
                    print(video_len, fps, begin_frame, end_frame)
                    break
                else:
                    continue
        
    # ------------------------------
    # 2. EXTRACT FRAMES AND SAVE AS A VIDEO
        
    if begin_frame > 0:

        cap = cv2.VideoCapture(original_dir + 'original_' + cat_name + '/' + video_name)
        count = begin_frame

        # cap의 시작 frame idx 설정
        cap.set(cv2.CAP_PROP_POS_FRAMES, begin_frame)

        # 코덱 설정
        fourcc = cv2.VideoWriter_fourcc(*'XVID')

        # frame 저장을 위한 영상 생성
        out1 = cv2.VideoWriter(target_dir + 'target_' + cat_name + '/color/' + action_name + video_name,\
                               fourcc, fps, (640, 360))
        out2 = cv2.VideoWriter(target_dir + 'target_' + cat_name + '/gray/' + action_name + video_name,\
                               fourcc, fps, (640, 360), isColor=False)

        # Extract 100 frames and save as a video
        while cap.isOpened() and count <= begin_frame + (how_many_frame-1):
            ret, frame = cap.read()

            if not ret:
                print(f'==ERROR: UNABLE TO READ FRAMES, VIDEO_NAME: {video_name}==')
                break

            frame = cv2.resize(frame, (640, 360))
#             cv2.imshow(video_name, frame)
            cv2.waitKey(25)

            out1.write(frame)

            # Get rid of color
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            out2.write(gray)

            # print extracted frame number
    #         if(int(cap.get(1)) % 1 == 0):
    #             print(f'Extracted frame number: {str(int(cap.get(1)))}')
            count += 1

            # 추출된 이미지 저장
    #         cv2.imwrite(frame_dir + f'{video_name.replace(".mp4", "")} + "_" + frame{count}" + ".jpg", frame')

        # 객체 해제
        cap.release()
        out1.release()
        out2.release()
        cv2.destroyAllWindows()
        
    else:
        print(f'==NO BEGIN FRAME FOUND, VIDEO_NAME: {video_name}==')
        continue

  7%|███                                        | 4/56 [00:56<12:38, 14.59s/it]

240.0 30 5740 5869


  9%|███▊                                       | 5/56 [01:09<12:01, 14.16s/it]

==NO BEGIN FRAME FOUND, VIDEO_NAME: 162-4_cam01_dump02_place04_night_summer.mp4==
==NO BEGIN FRAME FOUND, VIDEO_NAME: 162-4_cam02_dump02_place04_night_spring.mp4==


 16%|██████▉                                    | 9/56 [01:35<09:08, 11.67s/it]

...

 77%|████████████████████████████████▎         | 43/56 [06:17<02:33, 11.80s/it]

285.1 30 7161 7317


 89%|█████████████████████████████████████▌    | 50/56 [07:52<01:25, 14.33s/it]

==NO BEGIN FRAME FOUND, VIDEO_NAME: 164-3_cam02_dump02_place03_day_spring.mp4==


100%|██████████████████████████████████████████| 56/56 [09:02<00:00,  9.69s/it]
